In [1]:
# Install requirements
# to run the code, the following are required, and should be installed in a new environment:

# create new environment
%conda create -n "park_3.9" python=3.9 ipython -y
# activate environment
%conda activate park_3.9
# install package
%conda install numpy
# install requirements.txt
%pip install -r requirements.txt --verbose
# install ipykernel
%conda install -n park_3.9 ipykernel --update-deps --force-reinstall

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/peshangalo/opt/anaconda3/envs/park_3.9

  added / updated specs:
    - ipython
    - python=3.9


The following NEW packages will be INSTALLED:

  appnope            pkgs/main/osx-64::appnope-0.1.2-py39hecd8cb5_1001 None
  asttokens          pkgs/main/noarch::asttokens-2.0.5-pyhd3eb1b0_0 None
  backcall           pkgs/main/noarch::backcall-0.2.0-pyhd3eb1b0_0 None
  ca-certificates    pkgs/main/osx-64::ca-certificates-2022.10.11-hecd8cb5_0 None
  certifi            pkgs/main/osx-64::certifi-2022.9.24-py39hecd8cb5_0 None
  decorator          pkgs/main/noarch::decorator-5.1.1-pyhd3eb1b0_0 None
  executing          pkgs/main/noarch::executing-0.8.3-pyhd3eb1b0_0 None
  ipython            pkgs/main/osx-64::ipython-8.6.0-py39hecd8cb5_0

In [ ]:
# Code: https://github.com/deepakat002/numberplaterecognition
# Model(best.pt): https://github.com/pracool/ANPR-with-YoloV5

# %pip install opencv-python-headless==4.1.2.30

# %pip install --upgrade pip
# %pip install opencv-python
# %pip install matplotlib
# %pip install numpy
# %pip install pytesseract
# %pip install easyocr    
# %pip install tesseract

# %pip install seaborn

# %pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu # To install pytorch 

# !pip install -r requirements.txt


# download yolov5 from github, then in terminal go to the folder and reun "pip install -r requirements.txt"

In [27]:
# %pip install opencv-python
# %pip install pytesseract
# %pip install easyocr
# %pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu 
# %pip install pandas
# %pip install -r requirements.txt
# %pip install -r requirements_yolo.txt
!pip install -r requirements.txt

# https://towardsdatascience.com/getting-started-with-tesseract-part-ii-f7f9a0899b3f # for preprocess the image to improve accuracy of Pytesseract

  Cloning https://github.com/samson-wang/cython_bbox.git (to revision 9badb346a9222c98f828ba45c63fe3b7f2790ea2) to /private/var/folders/kh/s2d8ky754ks2jgqfq_m95dxm0000gn/T/pip-req-build-fv44swse
  Running command git clone --filter=blob:none --quiet https://github.com/samson-wang/cython_bbox.git /private/var/folders/kh/s2d8ky754ks2jgqfq_m95dxm0000gn/T/pip-req-build-fv44swse
  Running command git rev-parse -q --verify 'sha^9badb346a9222c98f828ba45c63fe3b7f2790ea2'
  Running command git fetch -q https://github.com/samson-wang/cython_bbox.git 9badb346a9222c98f828ba45c63fe3b7f2790ea2
  Running command git checkout -q 9badb346a9222c98f828ba45c63fe3b7f2790ea2
  Resolved https://github.com/samson-wang/cython_bbox.git to commit 9badb346a9222c98f828ba45c63fe3b7f2790ea2
  Preparing metadata (setup.py) ... done
  Using cached GitPython-3.1.29-py3-none-any.whl (182 kB)
  Using cached easydict-1.10-py3-none-any.whl
  Using cached gdown-4.5.4-py3-none-any.whl (14 kB)
  Using cached lap-0.4.0.tar.gz 

In [28]:
import easyocr

In [29]:
# importing required libraries
import cv2
import numpy as np
import pytesseract
import torch
from datetime import datetime
import csv
import os
import pandas as pd


In [30]:
import matplotlib.pyplot as plt
from PIL import Image

In [31]:
# DEFINING GLOBAL VARIABLE
EASY_OCR = easyocr.Reader(['en'])  # initiating easyocr
OCR_TH = 0.2

# Global Variables for text regecnitioin
SCREEN_WIDTH = 128
SCREEN_HEIGHT = 64


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [32]:
# -------------------------------------- function to run detection ---------------------------------------------------------
def detectx(frame, model):
    frame = [frame]
    print(f"[INFO] Detecting. . . ")
    results = model(frame)

    # results.show() # show image with plate detected as well as the cofidence
    # print( results.xyxyn[0])
    # print(results.xyxyn[0][:, -1])
    # print(results.xyxyn[0][:, :-1])

    labels, cordinates = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]


    return labels, cordinates

In [33]:
# ------------------------------------ to plot the BBox and results --------------------------------------------------------


def plot_boxes(results, frame, classes):
    """
    --> This function takes results, frame and classes
    --> results: contains labels and coordinates predicted by model on the given frame
    --> classes: contains the string labels
    """
    labels, cord = results
    n = len(labels)
    x_shape, y_shape = frame.shape[1], frame.shape[0]

    print(f"[INFO] Total {n} detections. . . ")
    print(f"[INFO] Looping through all detections. . . ")

    # looping through the detections
    for i in range(n):
        row = cord[i]
        if row[4] >= 0.55:  # threshold value for detection. We are discarding everything below this value
            print(f"[INFO] Extracting BBox coordinates. . . ")
            x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(
                row[2]*x_shape), int(row[3]*y_shape)  # BBOx coordniates
            text_d = classes[int(labels[i])]
            # cv2.imwrite("./output/dp.jpg",frame[int(y1):int(y2), int(x1):int(x2)])

            coords = [x1, y1, x2, y2]
            xmin, ymin, xmax, ymax = coords
            nplate = frame[int(ymin):int(ymax), int(xmin):int(xmax)]
            area = nplate.shape[0] * nplate.shape[1]
            #if area >=3000 and smaler that 4000
            if area > 3:
                plate_num = recognize_plate_easyocr(
                    img=frame, coords=coords, reader=EASY_OCR, region_threshold=OCR_TH)

                # if text_d == 'mask':
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # BBox
                # for text label background
                cv2.rectangle(frame, (x1, y1-20), (x2, y1), (0, 255, 0), -1)
                if(plate_num!= None):
                    cv2.putText(frame, f"{plate_num}", (x1, y1),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                    return frame
                else:
                    return None
                # cv2.imwrite("./output/np.jpg",frame[int(y1)-25:int(y2)+25, int(x1)-25:int(x2)+25])



In [34]:

# ---------------------------- function to recognize license plate --------------------------------------


# function to recognize license plate numbers using Tesseract OCR
def recognize_plate_easyocr(img, coords, reader, region_threshold):
    # separate coordinates from box
    xmin, ymin, xmax, ymax = coords
    # get the subimage that makes up the bounded region and take an additional 5 pixels on each side
    #nplate = img[int(ymin)+15:int(ymax)-5, int(xmin)+12:int(xmax)-12]
    # cropping the number plate from the whole image

    nplate = img[int(ymin):int(ymax), int(xmin):int(xmax)]
    # delete the left side with 10 px of the image
    nplate = nplate[:, 35:]

    # convert nplate to gray
    nplate = cv2.cvtColor(nplate, cv2.COLOR_BGR2GRAY)
    display(Image.fromarray(nplate))
    #print(nplate.shape)
    
    scale_factor = 6
    scaled_img = cv2.resize(nplate[1:50, 0:SCREEN_WIDTH], (0,0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
    assert(scaled_img is not None)
    #display(Image.fromarray(scaled_img))

    thres,thres_img = cv2.threshold(nplate, 0, 255, cv2.THRESH_BINARY|cv2.THRESH_OTSU)
    assert(thres_img is not None)
    #display(Image.fromarray(thres_img))

    ############################ Using Pytesseract to extract text from image ########################################
    text_1 = pytesseract.image_to_string(thres_img, config='--user-words words.txt config.txt')
    print('Text prediciton using Pytesseract:"{}"'.format(text_1)) 
    #############################################################################################################
    # write text_1 to file
    with open("output.txt", "a") as f:
        f.write(text_1)
        f.write("\n")
        f.close

    ####################################### using EasyORC to extract text ###############################################################
    area = nplate.shape[0] * nplate.shape[1]
    #if area >=3000 and smaler that 4000
    ocr_result = reader.readtext(nplate)
    text = filter_text(region=nplate, ocr_result=ocr_result,
                        region_threshold=region_threshold)
    if len(text) == 1:
        text = text[0].upper()
        print("Text prediction using EasyOCR: ", text)
        return text
    else:
        return None
    #############################################################################################################
    
   


In [35]:
# to filter out wrong detections

def filter_text(region, ocr_result, region_threshold):
    rectangle_size = region.shape[0]*region.shape[1]

    plate = []
    print(ocr_result)
    # write text to a file

        
    for result in ocr_result:
        length = np.sum(np.subtract(result[0][1], result[0][0]))
        height = np.sum(np.subtract(result[0][2], result[0][1]))

        if length*height / rectangle_size > region_threshold:
            plate.append(result[1])
    # if len(plate) == 1:
    #     with open("output.txt", "a") as f:
    #         f.write(str(plate[0]))
    #         f.write("\n")
    #         f.write(str(ocr_result[0][2]))
    #         f.write("\n")
    #         f.close
    return plate


In [36]:
# ---------------------------------------------- Main function -----------------------------------------------------

def main(img_path=None, vid_path=None, vid_out=None):

    print(f"[INFO] Loading model... ")
    # loading the custom trained model
    # model =  torch.hub.load('ultralytics/yolov5', 'custom', path='last.pt',force_reload=True) ## if you want to download the git repo and then run the detection
    model = torch.hub.load('./yolov5-master', 'custom', source='local',
                           path='best_submission.pt', force_reload=True)  # The repo is stored locally

    classes = model.names  # class names in string format

    # --------------- for detection on image --------------------
    if img_path != None:
        print(f"[INFO] Working with image: {img_path}")
        img_out_name = f"./output/result_{img_path.split('/')[-1]}"

        frame = cv2.imread(img_path)  # reading the image

        results = detectx(frame, model=model)  # DETECTION HAPPENING HERE

        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        frame = plot_boxes(results, frame, classes=classes)
        if frame is not None:
            # creating a free windown to show the result
            # display(Image.fromarray(frame))
            # cv2.namedWindow("img_only", cv2.WINDOW_NORMAL)
                

            if cv2.waitKey(5) & 0xFF == ord('q'):
                print(f"[INFO] Exiting. . . ")

            # if you want to save he output result.
            cv2.imwrite(f"{img_out_name}", frame)
            # stop from running the code
            return
    
        else:
            print(f"[INFO] Exiting. . . ")
            print("No plate detected in the image")
            return

    # --------------- for detection on video --------------------
    elif vid_path != None:
        print(f"[INFO] Working with video: {vid_path}")

        # reading the video
        cap = cv2.VideoCapture(vid_path)

        if vid_out:  # creating the video writer if video output path is given

            # by default VideoCapture returns float instead of int
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            codec = cv2.VideoWriter_fourcc(*'mp4v')  # (*'XVID')
            out = cv2.VideoWriter(vid_out, codec, fps, (width, height))

        # assert cap.isOpened()
        frame_no = 1
    
        # cv2.namedWindow("vid_out", cv2.WINDOW_NORMAL)
        while True:
            # start_time = time.time()
            ret, frame = cap.read()
            if ret and frame_no % 1 == 0:
                print(f"[INFO] Working with frame {frame_no} ")

                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = detectx(frame, model=model)
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

                frame = plot_boxes(results, frame, classes=classes)

                # display(Image.fromarray(frame))
                if vid_out:
                    print(f"[INFO] Saving output video. . . ")
                    out.write(frame)

                # if cv2.waitKey(5) & 0xFF == ord('q'):
                #     break
                # if cv2.waitKey(5) & 0xFF == ord('w'):
                #     get_max_accuracy()
                #     break
                frame_no += 1

        print(f"[INFO] Clening up. . . ")
        # releaseing the writer
        out.release()


# -------------------  calling the main function-------------------------------

In [37]:
# -------------------  calling the main function-------------------------------


# main(vid_path="./test_images/Parking_gate.mp4",vid_out="vid_1.mp4") ### for custom video
# main(vid_path=0,vid_out="webcam_facemask_result.mp4") #### for webcam
# read the image

# display(Image.fromarray(image))
main(img_path="11.png")   # for image

YOLOv5 🚀 2022-11-18 torch 1.13.0 CPU



[INFO] Loading model... 


Fusing layers... 
Model Summary: 232 layers, 7246518 parameters, 0 gradients
Adding AutoShape... 


[INFO] Working with image: 11.png
[INFO] Detecting. . . 


AttributeError: 'Upsample' object has no attribute 'recompute_scale_factor'

In [30]:
import cv2
import numpy as np
from threading import Thread
from time import sleep



class PlateExtraction:
    def __init__(self, image = None):
        self.image = image
        self.cascade_file = "cascade.xml"
        self.cascade = cv2.CascadeClassifier(self.cascade_file)
        self.minNeighbors = 1
        self.scaleFactor = 1.1#1.08
        self.thread = None
        self.plates = None



    def extract(self):
        """Extracts license plates from an image"""
        while True:
            if isinstance(self.image, np.ndarray):
                self.plates = self.cascade.detectMultiScale(self.image, self.scaleFactor, self.minNeighbors)
            else:
                pass
                sleep(0.1)

    def get_plates(self):
        """Returns the detected license plates"""
        return self.plates

    def run(self):
        """Runs the image processing in a thread"""
        self.thread = Thread(target=self.extract)
        # Setting it as a daemon so it terminates when the app thread terminates
        self.thread.daemon = True
        self.thread.start()

image = cv2.imread("1.jpeg")
plate_extraction = PlateExtraction(image)
plate_extraction.extract()
plate_extraction.run()
print(plate_extraction.get_plates())



KeyboardInterrupt: 

: 

In [83]:
import multiprocessing

def double(a):
    return a * 2


def driver_func():
    PROCESSES = 2
    with multiprocessing.Pool(PROCESSES) as pool:
        params = [(1, ), (2, ), (3, ), (4, )]
        results = [pool.apply_async(double, p) for p in params]

        for r in results:
            print('\t', r.get())


In [85]:
import multi
# driver_func()

In [86]:
multi.driver()

	 2
	 4
	 6
	 8


In [ ]:
# -------------------  calling the main function-------------------------------


# main(vid_path="./test_images/Parking_gate.mp4",vid_out="vid_1.mp4") ### for custom video
# main(vid_path=0,vid_out="webcam_facemask_result.mp4") #### for webcam

main(img_path="./Cars53.jpg")   # for image

#main thakes in a list of images and a boolean called drive_in

main(["./Cars53.jpg", "./Cars54.jpg", "./Cars55.jpg"], drive_in=True)

In [ ]:
# -------------------  calling the main function-------------------------------


main(vid_path="./Parking_gate.mp4",vid_out="vid_1.mp4") ### for custom video
# main(vid_path=0,vid_out="webcam_facemask_result.mp4") #### for webcam

# main(img_path="./test2.png")   # for image

In [69]:
string1 = "PD32939"
string2 = "PD22939"
string3 = "PD32248"
strings = [string1, string2, string3]
def filter_plate_text(strings):
    result = []
    for i in range(0, len(strings[0])):
        if strings[0][i] == strings[1][i] or strings[0][i] == strings[2][i]:
            result.append(strings[0][i])
        elif strings[1][i] == strings[2][i]:
            result.append(strings[1][i])
    # merge the list to a string
    result = "".join(result)
    print(result)

[['PD32939', 'PD22939', 'PD32248'], True]


In [39]:
from datetime import datetime
import csv
import os
import pandas as pd

In [36]:
def write_to_csv_drive_in(_plate_number):   
    # Headrs for the csv files
    drive_in_colums_headers = ['Plate Number', 'Date and time']
    drive_out_colums_headers = ['Plate Number', 'Drive in time', 'Drive out time', 'Duration']

    # This will change with accual value 
    plate_number = _plate_number

    #########  get the current date and time #######
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    #############################################



    # if the header is already written, then don't write it again. If not, then write it
    if not os.path.exists('drive_in.csv'):
        with open('drive_in.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerow(drive_in_colums_headers)
            writer.writerow([plate_number,dt_string])
    else:
        with open('drive_in.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([plate_number,dt_string])

    if not os.path.exists('drive_out.csv'):
        with open('drive_out.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerow(drive_out_colums_headers)
            writer.writerow([plate_number,dt_string])
    else:
        with open('drive_out.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([plate_number,dt_string])
    return


In [48]:
def delete_row_from_csv(_plate_number, file_name):
    df = pd.read_csv(file_name+'.csv')
    df = df[df['Plate Number'] != _plate_number]
    df.to_csv('drive_in.csv', index=False)
    return

In [62]:

def write_to_csv_drive_out(_plate_number):
    # This will change with accual value
    plate_number = _plate_number

    #########  get the current date and time####
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    #############################################

    # read the csv file
    df = pd.read_csv('drive_out.csv')

    # find the row with the plate number where the drive out time is empty
    row = df.loc[(df['Plate Number'] == plate_number) & (df['Drive out time'].isnull())]

    # get the index of the row
    index = row.index[0]

    # update the drive out time for the plate number
    df.at[index, 'Drive out time'] = dt_string

    # calculate the duration
    drive_in_time = df.at[index, 'Drive in time']
    drive_out_time = df.at[index, 'Drive out time']
    drive_in_time = datetime.strptime(drive_in_time, "%d/%m/%Y %H:%M:%S")
    drive_out_time = datetime.strptime(drive_out_time, "%d/%m/%Y %H:%M:%S")
    duration = drive_out_time - drive_in_time

    # update the duration
    df.at[index, 'Duration'] = duration

    # Drop the row with the plate number from the drive_in.csv file
    delete_row_from_csv(plate_number, 'drive_in')

    # save the updated csv file
    df.to_csv('drive_out.csv', index=False)

    # print the updated csv file
    print(df)
    return



In [68]:
#write_to_csv_drive_in("ABC123")
write_to_csv_drive_out("ABC123")

  Plate Number        Drive in time       Drive out time Duration
0       ABC123  02/12/2022 11:12:10  02/12/2022 11:24:38  0:12:28
1       ABC123  02/12/2022 11:19:47  02/12/2022 11:25:37  0:05:50
2       ABC123  02/12/2022 11:25:49  02/12/2022 11:25:58  0:00:09
3       ABC123  02/12/2022 11:29:00  02/12/2022 11:29:16  0:00:16


In [ ]:

def main2(images, drive_in):
    output = []
    # loop through the three images
    for i in range(3):
        # print "working with image number: ", i
        print("working with image number: ", i)
        # call the main function
        plate_num = main(images[i])
        # remve the space in the plate number
        plate_num = plate_num.replace(" ", "")
        # save the output to a list
        output.append(plate_num)
    

    print("output: ", output)



    if drive_in:
        print(f"the car with licence palte {print(max(set(output), key=output.count))} driving in")
    else:
        print(f"the car with licence palte {print(max(set(output), key=output.count))} driving out")



In [70]:
image1 = cv2.imread("11.png")
image2 = cv2.imread("22.png")
image3 = cv2.imread("33.png")
images = [image1, image2, image3]

alex_array = []
# add strings array to alex_array
alex_array.append(images)
alex_array.append(True)

# seperate alex_array to images and drive_in
images, drive_in = alex_array

[[array([[[ 72,  86,  94],
        [ 72,  86,  94],
        [ 72,  86,  94],
        ...,
        [ 49,  55,  63],
        [ 61,  67,  75],
        [ 77,  83,  93]],

       [[ 72,  87,  95],
        [ 72,  86,  95],
        [ 71,  87,  94],
        ...,
        [ 46,  53,  60],
        [ 66,  70,  78],
        [ 78,  85,  94]],

       [[ 73,  85,  97],
        [ 72,  85,  96],
        [ 72,  85,  97],
        ...,
        [ 46,  53,  61],
        [ 71,  79,  88],
        [ 81,  92, 100]],

       ...,

       [[ 42,  50,  54],
        [ 41,  49,  54],
        [ 39,  48,  52],
        ...,
        [ 56,  59,  61],
        [ 57,  58,  63],
        [ 57,  57,  64]],

       [[ 42,  48,  53],
        [ 41,  49,  53],
        [ 41,  48,  52],
        ...,
        [ 57,  57,  60],
        [ 57,  57,  62],
        [ 56,  57,  63]],

       [[ 41,  49,  53],
        [ 41,  48,  54],
        [ 40,  48,  52],
        ...,
        [ 57,  59,  61],
        [ 57,  58,  63],
        [ 57,  57,  63